In [1]:
import sys
sys.path.append('..')

In [2]:
import sentence_embedding_evaluation_german as seeg
import torch
import numpy as np
from torch_emb2vec import ConvToVec
from typing import List

## (1) Instantiate your Embedding model
First, you should load your pretrained embedding.

Here we will generate a random embedding for demonstration purposes.

In [3]:
# generate a random embedding
emb_dim = 256
vocab_sz = 128
emb = torch.randn((vocab_sz, emb_dim), requires_grad=False)
emb = torch.nn.Embedding.from_pretrained(emb)
# assert emb.weight.requires_grad == False

## (2a) Specify the preprocessing
The `preprocessor` function converts a sentences as string into embedding vectors of numbers.

Here we will convert the input strings with a nonsensical approach into IDs for the Embedding layer.

In [4]:
SEQLEN = 128
padid = 128

def preprocesser(batch: List[str], params: dict=None) -> List[List[List[float]]]:
    """ Specify your embedding or pretrained encoder here
    Paramters:
    ----------
    params : dict
        The params dictionary
    batch : List[str]
        A list of sentence as string
    Returns:
    --------
    List[List[List[float]]]
        A list of embedded sequences
    """
    features = []
    for sent in batch:
        # encode
        try:
            ids = torch.tensor([ord(c) % 128 for c in sent])
        except:
            print(sent)
        enc = emb(ids)
        # truncate & pad
        h = torch.ones(size=(SEQLEN, enc.shape[1]), dtype=torch.int64) * padid
        end = min(enc.shape[0], SEQLEN)
        try:
            h[:end, :] = enc[:end, :].detach()
        except Exception as e:
            raise Exception(e)
        features.append(h)
    features = torch.stack(features, dim=0)
    features = features.type(torch.float32)
    return features.detach()

## (2b) Specify a Customer Classifier

In [5]:
class CustomClassiferModel(torch.nn.Module):
    def __init__(self,
                 embdim: int,
                 output_size: int,  # nclasses
                 *args, **kwargs):
        super(CustomClassiferModel, self).__init__(*args, **kwargs)
        # Self-MHA layer (=> this will overfit bcoz tiny datasets!)
        #self.mha_pre = torch.nn.LayerNorm(embdim, elementwise_affine=True)
        #self.mha_net = torch.nn.MultiheadAttention(
        #    embed_dim=embdim, num_heads=8, batch_first=True, bias=False)
        # 2D to 1D flattening
        self.to1d_pre = torch.nn.LayerNorm(embdim, elementwise_affine=True)
        # compute kernel size and output channel dim
        self.to1d_net = ConvToVec(
            seq_len=SEQLEN, emb_dim=embdim, num_output=256, trainable=False)
        self.to1d_act = torch.nn.GELU()
        # Final layer
        self.final_pre = torch.nn.LayerNorm(256, elementwise_affine=True)
        self.final_net = torch.nn.Linear(256, output_size, bias=False)
        self.final_act = torch.nn.Softmax(dim=1)
        # init params
        self._reset_parameters()

    def _reset_parameters(self) -> None:
        torch.manual_seed(42)
        # Self-MHA projections
        #for param in self.mha_net.parameters():
        #    torch.nn.init.xavier_normal_(param, gain=1.0)
        # Final layer
        torch.nn.init.xavier_normal_(self.final_net.weight, gain=1.0)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        # Self-MHA layer
        #print("Inputs", inputs.shape)
        h = inputs
        #h = self.mha_pre(h)
        #h, _ = self.mha_net(query=h, value=h, key=h)
        #h = h + inputs  # skip-conn
        # print("MHA", h.shape)
        # 2D to 1D flattening
        h = self.to1d_pre(h)
        h = self.to1d_net(h)
        h = self.to1d_act(h)
        #print("Flatten", h.shape)
        # Final layer
        h = self.final_pre(h)
        h = self.final_net(h)
        #print("Final", h.shape)
        return self.final_act(h)


def mymodel(**kwargs):
    return CustomClassiferModel(
        embdim=kwargs['n_features'],
        output_size=kwargs['n_classes'])


In [6]:
model = mymodel(n_features=emb_dim, n_classes=3)
n = sum([m.numel() for m in model.parameters()])
print(f"Num params: {n}")

Num params: 68352


## (3) Training settings

In [7]:
params = {
    'datafolder': '../datasets',
    'bias': True,
    'balanced': True,
    'batch_size': 128, 
    'num_epochs': 5,  # Default: 500
    # 'early_stopping': True,
    # 'split_ratio': 0.2,  # if early_stopping=True
    # 'patience': 5,  # if early_stopping=True
}

## (4) Specify downstream tasks

In [8]:
# All
# downstream_tasks = [
#     'TOXIC', 'ENGAGE', 'FCLAIM', 'VMWE',
#     'OL19-A', 'OL19-B', 'OL19-C',
#     'OL18-A', 'OL18-B', 
#     'ABSD-1', 'ABSD-2', 'ABSD-3',
#     'MIO-S', 'MIO-O', 'MIO-I', 'MIO-D', 'MIO-F', 'MIO-P', 'MIO-A',
#     'SBCH-L', 'SBCH-S', 'ARCHI', 'LSDC'
# ]

# Group tasks
# downstream_tasks = [
#     'ABSD-2', 'MIO-S', 'SBCH-S',  # Sentiment analysis
#     'ENGAGE', 'MIO-P',  # engaging/personal
#     'FCLAIM', 'MIO-A',  # fact-claim (potential fake news), argumentative, reasoning
#     'TOXIC', 'OL19-A', 'OL19-B', 'OL19-C', 'MIO-O', 'MIO-I',  # toxic
# ]

# Current favorites
downstream_tasks = ['FCLAIM', 'VMWE', 'OL19-C', 'ABSD-2', 'MIO-P', 'ARCHI', 'LSDC']
downstream_tasks = ['FCLAIM', 'VMWE', 'OL19-C', 'ABSD-2', 'MIO-P', 'ARCHI']

## (5) Run experiments

In [9]:
%%time
results = seeg.evaluate(
    downstream_tasks=downstream_tasks, 
    preprocesser=preprocesser, 
    modelbuilder=mymodel,
    verbose=1,
    **params
)

Dataset loaded: FCLAIM
epoch 1 | loss: 0.714004798577382
epoch 2 | loss: 0.6992716720471015
epoch 3 | loss: 0.6866543568097628
epoch 4 | loss: 0.6757772335639367
epoch 5 | loss: 0.6673626349522517
Dataset loaded: VMWE
epoch 1 | loss: 0.7426156963293369
epoch 2 | loss: 0.7321090480455985
epoch 3 | loss: 0.7239403472496913
epoch 4 | loss: 0.717761639219064
epoch 5 | loss: 0.7133696629450872
Dataset loaded: OL19-C
epoch 1 | loss: 0.7187334559857845
epoch 2 | loss: 0.7131192833185196
epoch 3 | loss: 0.7092077378183603
epoch 4 | loss: 0.7055755704641342
epoch 5 | loss: 0.7020894940942526
Dataset loaded: ABSD-2
epoch 1 | loss: 1.13924005078642
epoch 2 | loss: 1.1295296008649625
epoch 3 | loss: 1.1202633294620012
epoch 4 | loss: 1.111301327222272
epoch 5 | loss: 1.1033252940366143
Dataset loaded: MIO-P
epoch 1 | loss: 0.7111242767926809
epoch 2 | loss: 0.6952965533411181
epoch 3 | loss: 0.6815511358750833
epoch 4 | loss: 0.6689673919935484
epoch 5 | loss: 0.6584070647085035
Dataset loaded: AR

In [10]:
import json
dat = json.dumps(results, indent=2)
# print(dat)

## (6) Display results

In [11]:
print("Task | Epochs | N train | N test")
for res in results:
    print(f"{res['task']:>7s}: {res['epochs']:5d} {res['train']['num']:6d} {res['test']['num']:6d}")

Task | Epochs | N train | N test
 FCLAIM:     5   3244    944
   VMWE:     5   6652   1447
 OL19-C:     5   1921    930
 ABSD-2:     5  19432   2555
  MIO-P:     5   4668   4668
  ARCHI:     5  18809   4743


In [12]:
metric = 'acc'  # 'f1', 'f1-balanced', 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.639  0.606
   VMWE:  0.675  0.679
 OL19-C:  0.536  0.528
 ABSD-2:  0.382  0.359
  MIO-P:  0.597  0.581
  ARCHI:  0.282  0.253


In [13]:
metric = 'f1-balanced'  # 'f1', 'f1-balanced', 'acc', 'acc-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.606  0.576
   VMWE:  0.505  0.502
 OL19-C:  0.445  0.445
 ABSD-2:  0.317  0.290
  MIO-P:  0.534  0.512
  ARCHI:  0.275  0.249


In [14]:
# class label distributions (inference)
for res in results:
    print(res['task'], res['test']['num'])
    print(res['test']['distr-test'])
    print(res['test']['distr-pred'])

FCLAIM 944
{'0': '630', '1': '314'}
{'1': '378', '0': '566'}
VMWE 1447
{'1': '256', '0': '1191'}
{'0': '1119', '1': '328'}
OL19-C 930
{'0': '796', '1': '134'}
{'0': '493', '1': '437'}
ABSD-2 2555
{'1': '1670', '0': '780', '2': '105'}
{'1': '972', '2': '841', '0': '742'}
MIO-P 4668
{'1': '812', '0': '3856'}
{'0': '2561', '1': '2107'}
ARCHI 4743
{'2': '1177', '3': '1176', '1': '1199', '0': '1191'}
{'1': '1711', '2': '1034', '3': '975', '0': '1023'}


In [15]:
# class label distribution (training)
for res in results:
    print(res['task'], res['train']['num'])
    print(res['train']['distr-train'])
    print(res['train']['distr-pred'])

FCLAIM 3244
{'0': '2141', '1': '1103'}
{'1': '1207', '0': '2037'}
VMWE 6652
{'1': '1145', '0': '5507'}
{'1': '1607', '0': '5045'}
OL19-C 1921
{'0': '1664', '1': '257'}
{'0': '1032', '1': '889'}
ABSD-2 19432
{'2': '1179', '1': '13208', '0': '5045'}
{'2': '6412', '0': '5424', '1': '7596'}
MIO-P 4668
{'0': '3855', '1': '813'}
{'0': '2535', '1': '2133'}
ARCHI 18809
{'1': '4797', '3': '4407', '2': '4802', '0': '4803'}
{'3': '3578', '2': '3626', '1': '7865', '0': '3740'}
